Vector Autoregressions
## Vector Autoregresion (VAR) Estimation

**Functions**

`tsa.VAR`

### Exercise 85
Download data on 10-year interest rates, 1-year interest rates and the
GDP deflator from FRED.

In [ ]:
import pandas as pd

gs1 = pd.read_csv("./data/GS1.csv", parse_dates=True)
gs1 = gs1.set_index("DATE")
gs10 = pd.read_csv("./data/GS10.csv", parse_dates=True)
gs10 = gs10.set_index("DATE")
defl = pd.read_csv("./data/GDPDEF.csv", parse_dates=True)
defl = defl.set_index("DATE")

data = pd.concat([gs1, gs10, defl], axis=1)
data.columns = ["gs1", "gs10", "defl"]
data.head(6)

#### Explanation

The data have all been downloaded from FRED and saved as csv files. The
series are imported and merged into a single DataFrame. We see that
the deflator is quarterly while the others are monthly.

In [ ]:
data.index = pd.to_datetime(data.index)
data = data.resample("Q").mean().dropna()
data.head()

#### Explanation
We can use `resample` to convert all of the series to quarterly to match the
deflator. The mean is a reasonable method to aggregate the interest rates and
since pandas ignores `NaN`, the mean of the deflator is the observation
available in each quarter.

### Exercise 86
Transform the GDP deflator to be percent returns (e.g. $\Delta\ln\left(GDP_t\right)$ ).

In [ ]:
import numpy as np

log_defl = np.log(data.defl)
data["deflg"] = log_defl - log_defl.shift(1)
data = data.dropna()
data.head()

#### Explanation

Here we use `np.log` and `shift` to implement the log difference.

### Exercise 87
Estimate a first-order VAR on the spread between the 10-year and 1-year
(spread), the one-year, and the growth rate of the GDP deflator.

In [ ]:
import statsmodels.tsa.api as tsa

data["spread"] = data.gs10 - data.gs1
# Save for later
data.to_hdf("./data/var-data.h5", "var_data")

#### Explanation

The spread is constructed as the difference and the data is saved for use in 
other exercises.

In [ ]:
mod = tsa.VAR(data[["spread", "gs1", "deflg"]])
res = mod.fit(1, trend="c", ic=None)

res.summary()

#### Explanation

A VAR model is specified using `tsa.VAR`. The only required input is the data.
We do not include the intercept since this is supplied through the `trend`
argument of `fit`, where "c" indicates a constant. We set the maximum
lag to 1 and `ic` to `None` to get force a VAR(1) to be estimated. If we
do not set `ic` to `None`, statsmdoels will perform a lag length search for
lags in 0, 1, ..., `maxlags` (0 or 1 in this specification).

### Exercise 88
What are the _own_ effects?


In [ ]:
res.params

In [ ]:
own_effects = {}
for var in res.params:
    own_effects[var] = res.params.loc[f"L1.{var}", var]
pd.DataFrame(pd.Series(own_effects, name="Own Effect"))

#### Explanation

The parameters are a `DataFrame` where the columns are the leads and the rows
are lag or trend terms. 

### Exercise 89
What are the cross effects between these?

In [ ]:
other_effects = {}
for var in res.params:
    for other in res.params:
        if other == var:
            continue
        other_effects[(var, other)] = res.params.loc[f"L1.{other}", var]

s = pd.Series(other_effects, name="effect")
s.index = s.index.set_names(["lead", "lag"])
pd.DataFrame(s)

#### Explanation

These effects are hard to interpret since the series have not been
standardized to have the same variance.

### Exercise 90
How could you get a sense of the persistence of this system?

In [ ]:
# Exclude the constant
phi = res.params.iloc[1:]
evals = np.linalg.eigvals(phi)
print(f"The maximum eigenval is {np.max(np.abs(evals))}")

#### Explanation

The maximum eigenvalue of the VAR(1) parameters provides a measure of the
persistence in the model. It is close to 1 so these values are highly
persistent.

In [ ]:
std_data = data / data.std()
mod = tsa.VAR(std_data[["spread", "gs1", "deflg"]])
res = mod.fit(1, trend="c", ic=None)

res.params.iloc[1:]

In [ ]:
phi = res.params.iloc[1:]
evals = np.linalg.eigvals(phi)
print(f"The maximum eigenval is {np.max(np.abs(evals))}")

#### Explanation

We repeat the exercise using data standardized by their standard deviations.
While the coefficients change (except own effects), the eigenvalues are unaffected. 
The coefficients are directly interpretable in terms of a 1 standard deviation change
in each variable.